**Data Science and AI for Energy Systems** 

Karlsruhe Institute of Technology

Institute of Automation and Applied Informatics

Summer Term 2024

---

# Exercise X: Forecasting

**Imports**

In [ ]:
import pandas as pd
import torch
from torch import nn

## Problem X.2 (programming) - Load Forecasting

i. First let us quickly validate our results from exercise X.1 using the LSTM cell implementation in PyTorch.

In [ ]:
# As all our biases are zero we can set bias=False
lstm = nn.LSTMCell(1,1, bias=False)

In [ ]:
# The wights of the LSTMCell are stored in lstm.wight_ih and lstm.weight_hh
print(lstm.weight_ih)
print(lstm.weight_hh)

In [ ]:
# we need to initialize all weights to 1 to match the LSTM from exercise X.1
# Therefore use nn.init.ones_() on all weights of the LSTMCell object (lstm.weight_ih, lstm.weight_hh)
nn.init.ones_(lstm.weight_ih)
nn.init.ones_(lstm.weight_hh)

print(lstm.weight_ih)
print(lstm.weight_hh)

In [ ]:
# Now initialize h, C and x as torch.tensor with the values from the exercise
h = torch.tensor([[0.0]])
C = torch.tensor([[-1.0]])
x = torch.tensor([[1.0]])

In [ ]:
# now compute the forward pass of the LSTMCell by calling your initialized LSTMCell object with the input x and the hidden states h and C
lstm(x, (h, C))

ii. Now we forecast electrical load using multiple deep learning (DL) forecasting algorithms by Neuralforecast utilizing this [dataset](https://data.open-power-system-data.org/time_series/2020-10-06/time_series_60min_singleindex.csv). The documentation for the dataset can be found [here](https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/2020-10-06/main.ipynb).

(a) We first load our dataset.

In [ ]:
#documentation: https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/2020-10-06/main.ipynb
#loading data:

# URL of the CSV file
url = "https://data.open-power-system-data.org/time_series/2020-10-06/time_series_60min_singleindex.csv"

# Load the dataset
data = pd.read_csv(url)

# Display the column names and data types
print("\nColumn names and data types:")
print(data.info())

(b) Then we format the data so that it follows the format required by the DL library we are using: Neural Forecast.

In [ ]:
#columns we need from the large dataset
target_var = 'DE_transnetbw_load_actual_entsoe_transparency'
timestamp = 'utc_timestamp'

#assigning a unique id for the time series
data['unique_id'] = 1
#changing format to datetime
data[timestamp] = pd.to_datetime(data[timestamp])
#setting date as index to facilitate selecting sets
data['ds'] = data[timestamp]
data.set_index(timestamp, inplace=True)
#localizing index for compatability:
data.index = data.index.tz_localize(None)

#selecting only the needed columns and ignoring first value as it doesn't have a value
data = data.loc['2015-01-01':, ['unique_id', 'ds', target_var]]

#changing column names to suit neuralforecast
data.rename(columns={target_var:'y'}, inplace=True)

data.head(3)

(c) We split the data into trainset and testset.

In [ ]:
train = data[:'2019']
test = data['2020':]

(d) We import the algorithms we are interested in, initialize the models with hyperparameters and then pass this to the training phase.

In [ ]:
from neuralforecast import NeuralForecast as NF
from neuralforecast.core import NeuralForecast as TNF
from neuralforecast.models import LSTM, NHITS, MLP

In [ ]:
horizon = 24
historical = 24 * 7
lstm_model = LSTM(h=horizon,                 # Forecast horizon
                  input_size=historical,        # length of previous series to use for forecast
                  max_steps=1000,               # Number of epochs to train
                  scaler_type='standard',       # Type of scaler to normalize data
                  encoder_n_layers=2,
                  encoder_hidden_size=100,      # Defines the size of the hidden state of the LSTM
                  decoder_hidden_size=512,
                  learning_rate=0.0003,
                  batch_size=32,
                 )

mlp_model = MLP(h=horizon,                 # Forecast horizon
                  input_size=historical,        # length of previous series to use for forecast
                  max_steps=100,               # Number of epochs to train
                  scaler_type='standard',       # Type of scaler to normalize data
                 )


#creating the model and specifying that we have hourly time series
nf = NF(models=[lstm_model, mlp_model], freq='h')

(e) We pass the training set to the model for training.

In [ ]:
nf.fit(train)

(f) We then use the model for predicting the next 24 hours given the previous week on the test set.

In [ ]:
daterange = pd.date_range('2020-01-01 00:00', '2020-09-23 00:00', freq='24h')

In [ ]:
predictions = [nf.predict(test.loc[d:d + pd.Timedelta(hours=7*24), :]) for d in daterange]

In [ ]:
predictions_df = pd.concat(predictions)

In [ ]:
predictions_df.index = predictions_df.ds
predictions_df.index = predictions_df.index.tz_localize(None)

In [ ]:
predictions_df = predictions_df.drop(['ds'], axis=1)

In [ ]:
results = pd.concat([test.y, predictions_df], axis = 1)

In [ ]:
# plot the predictions for the complete test set
results.plot()

In [ ]:
# plot the predictions for one month (e.g. July)
results['2020-07':'2020-07'].plot()

In [ ]:
# plot the predictions for one month (e.g. July)
results['2020-03':'2020-03'].plot()

In [ ]:
from neuralforecast.losses.numpy import mae
mae_lstm = mae(results['y'], results['LSTM'])
print("MAE of LSTM is: ", mae_lstm)

In [ ]:
mae_lstm = mae(results['y'], results['MLP'])
print("MAE of MLP is: ", mae_lstm)

In [ ]:
# print the losses for each weekday
wd_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

for i in range(7):
    wd_results = results[results.index.weekday == i]
    wd_mae = mae(wd_results['y'], wd_results['LSTM'])
    print("MAE of LSTM for ", wd_dict[i], " is: ", wd_mae)

(g) Now play a little with the hyperparameters for LSTM. Leave the forecasting horizon and input size as they are.
*optional:* You can play with AutoLSTM.

In [ ]:
from neuralforecast.auto import AutoLSTM